# Time resample

In [1]:
import pandas as pd
from read_json import as_json, format_time
import numpy as np
import matplotlib.pyplot as plt

In [15]:
data = pd.read_csv("data_samples/data_sample_4.csv")
data["@timestamp"] = pd.to_datetime(data["@timestamp"].map(format_time))  # convert to pandas datetime
id_nan =data.system.isna()
data_no_nan = data.loc[~id_nan]

In [10]:
hostnames = data["host"].unique()            # get unique names
hostnames = list(map(as_json, hostnames))
hostnames

[{'name': 'PRUEBAS'},
 {'name': 'PRODUCCION'},
 {'name': 'PREPRODUCCION'},
 {'name': 'QASERVER'},
 {'name': 'DEVELOP'}]

In [16]:
cpu_df = pd.read_csv("cpu_df.csv",index_col=0)
mem_df = pd.read_csv("memory_df.csv", index_col=0)
net_df = pd.read_csv("network_df.csv", index_col=0)

frames = [cpu_df, mem_df, net_df]
df = pd.concat(frames)  # si jala
df.sort_index(inplace=True)
#df["@timestamp"] = data_no_nan["@timestamp"]
df.insert(0, "@timestamp", data_no_nan["@timestamp"])
df_time = df

## DEVELOP
df_develop = df_time.loc[df_time["host"] == "DEVELOP"].drop("host", axis=1)
df_develop_resampled = df_develop.resample('10Min', on="@timestamp").mean()
df_develop_resampled["cores"] = 4
#df_develop_resampled.to_csv("resampled/develop_resampled.csv")

## QASERVER
# do it by host -->  then relate by host
df_qaserver = df_time.loc[df_time["host"] == "QASERVER"].drop("host", axis=1)
# df_time["@timestamp"].resample("5Min")
df_qaserver_resampled = df_qaserver.resample('10Min', on="@timestamp").mean()
df_qaserver_resampled["cores"] = 4  # the server has the same number of cores
#df_qaserver_resampled.to_csv("resampled/qaserver_resampled.csv")


## PRODUCCION
df_produccion = df_time.loc[df_time["host"] == "PRODUCCION"].drop("host", axis=1)
df_produccion_resampled = df_produccion.resample('10Min', on="@timestamp").mean()
df_produccion_resampled["cores"] = 8
#df_produccion_resampled.to_csv("resampled/produccion_resampled.csv")
df_produccion_resampled

## PREPRODUCCION
df_preproduccion = df_time.loc[df_time["host"] == "PREPRODUCCION"].drop("host", axis=1)
df_preproduccion_resampled = df_preproduccion.resample('10Min', on="@timestamp").mean()
df_preproduccion_resampled["cores"] = 4
#df_preproduccion_resampled.to_csv("resampled/preproduccion_resampled.csv")
df_preproduccion_resampled

## PRUEBAS
df_pruebas = df_time.loc[df_time["host"] == "PRUEBAS"].drop("host", axis=1)
df_pruebas_resampled = df_pruebas.resample('10Min', on="@timestamp").mean()
df_pruebas_resampled["cores"] = 16
#df_pruebas_resampled.to_csv("resampled/pruebas_resampled.csv")

data_full = pd.concat([df_develop_resampled,
                       df_produccion_resampled,
                       df_preproduccion_resampled,
                       df_pruebas_resampled,
                       df_qaserver_resampled])
data_full.sort_index(inplace=True)
data_full.to_csv("resampled/data_4_resampled.csv")